### **Course: Advanced Prompt Engineering for Banking**

### **Lab 4: Dynamic and Verifiable Prompting Frameworks**

**Objective:** This lab explores advanced frameworks that enable LLMs to interact with external systems and verify their own statements. You will learn to implement **ReAct (Reason and Act)** to build tool-using agents and **Chain-of-Verification (CoVe)** to enhance factual accuracy, both critical for creating reliable AI solutions in finance.

-----
### Created By: Prashant Sahu
[Connect with me on LinkedIn](https://www.linkedin.com/in/prashantksahu/)

-----
-----
</br>



### **Setup**

This lab requires the `langchain-community` and `langchainhub` libraries for agent creation.

```bash
pip install langchain langchain-openai python-dotenv langchain-community langchainhub
```

**lab\_script.py:**

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub

# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# Helper function for simple prompt execution (for CoVe)
def execute_prompt(prompt_template, title, **kwargs):
    print(f"--- {title} ---")
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke(kwargs)
    print("PROMPT:")
    print(prompt.format(**kwargs))
    print("\n" + "="*20 + " RESPONSE " + "="*20 + "\n")
    print(response)
    print("\n" + "-"*50 + "\n")
    return response

-----

## **1. ReAct (Reason and Act): Giving LLMs Tools**

### **Background**

**ReAct** is a paradigm that transforms a language model from a simple text generator into a reasoning agent that can interact with its environment. It synergizes reasoning (like in Chain-of-Thought) with the ability to take actions.

  * **How it works:** The LLM operates in a loop: **Thought** -\> **Action** -\> **Observation**.
    1.  **Thought:** The model thinks about the user's query and decides what it needs to do.
    2.  **Action:** It chooses a `tool` (e.g., a database query, an API call) and the `input` for that tool.
    3.  **Observation:** The tool executes, and its output is fed back to the model as an observation.
        This loop continues until the model has enough information to form a final answer.
  * **Strengths:** Allows LLMs to overcome their inherent limitations, such as knowledge cutoffs and the inability to access private data. It makes them dynamic problem-solvers that can fetch real-time information.
  * **Banking Use Cases:** Checking live stock prices, fetching a customer's account balance from a secure database, calculating loan EMIs using a precise financial calculator, or looking up current forex rates before executing a trade.

### **Use Case 1.1: Customer Account & Transaction Inquiry**

**Scenario:** A customer service chatbot needs to answer a specific question about a customer's account balance and recent transactions.

#### **Naive Prompt (Non-ReAct) 👎**

The model has no access to real-time, private data. It will either refuse to answer or, worse, **hallucinate** an answer.

In [ ]:
# This is a simple execution without any tools
naive_prompt_react_1 = "Hi, can you tell me the current balance for account number 1002345 and list my last 2 transactions?"

# execute_prompt(
#     "{query}",
#     "1.1 Naive Account Inquiry",
#     query=naive_prompt_react_1
# )

**Expected Response:**

```
I am an AI model and do not have access to real-time, private banking information. To get your account balance and transaction history, please log in to your official banking app or contact customer support.
```

This is a safe but unhelpful response.

#### **ReAct Implementation 👍**

Here, we define mock "tools" that the LLM can use. In a real application, these would be secure API calls to the bank's core systems.

In [ ]:
# --- Tool Definition ---
# We define simple Python functions as our tools.
@tool
def get_account_balance(account_id: str) -> str:
    """
    Returns the current balance for a given bank account ID.
    Use this tool to find a customer's account balance.
    """
    print(f"--- TOOL EXECUTED: get_account_balance({account_id}) ---")
    # Mock data for demonstration
    balances = {"1002345": "₹1,50,234.50", "9876543": "₹8,750.00"}
    return balances.get(account_id, "Account not found.")

@tool
def get_last_n_transactions(account_id: str, n: int) -> str:
    """
    Returns the last 'n' transactions for a given bank account ID.
    Use this tool to find a customer's recent transaction history.
    """
    print(f"--- TOOL EXECUTED: get_last_n_transactions({account_id}, {n}) ---")
    transactions = {
        "1002345": [
            "2025-09-15 | DEBIT | ₹2,500.00 | Merchant: SWIGGY",
            "2025-09-14 | DEBIT | ₹12,000.00 | Merchant: AMAZON",
            "2025-09-12 | CREDIT | ₹75,000.00 | Source: SALARY CREDIT"
        ]
    }
    return str(transactions.get(account_id, "Account not found.")[:n])

# --- Agent Creation ---
# Pull the ReAct prompt template
prompt = hub.pull("hwchase17/react")

# List the tools the agent can use
tools = [get_account_balance, get_last_n_transactions]

# Create the agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Agent Execution ---
# query = "Hi, can you tell me the current balance for account number 1002345 and list my last 2 transactions?"
# response = agent_executor.invoke({"input": query})
# print("--- FINAL AGENT RESPONSE ---")
# print(response['output'])

### **ReAct Technique vs. LangChain's ReAct Agent**

#### Question: Is ReAct Technique same as creating a ReAct Agent?
No, the **ReAct prompting technique** and creating a **ReAct agent in LangChain** are not exactly the same, but they are directly related.

  * **ReAct Technique:** This is the **conceptual framework** first described in a research paper. It establishes the idea that a language model can achieve complex tasks by interleaving `Reasoning` (thinking about what to do) and `Acting` (using a tool to get external information). It's a blueprint for how an agent should think.

  * **LangChain ReAct Agent:** This is a specific, practical **implementation** of the ReAct technique. LangChain provides the scaffolding (code, prompts, and parsers) to make an LLM follow the "Thought -\> Action -\> Observation" loop easily. It's the ready-to-use engine built according to the ReAct blueprint.

#### Question: Can we implement the ReAct Technique without creating a ReAct Agent?
**No, you cannot meaningfully implement the ReAct framework without tools.** The **"Act"** part of ReAct is its entire purpose. It's what separates it from a simple Chain-of-Thought prompt. The action is always the use of a tool, whether that tool is a web search, a database query, a calculator, or a call to another API. Without a tool to "Act" upon, the framework collapses into just a "Reason" step, which is essentially Chain-of-Thought.

-----


### **ReAct Use Case 1.2: Real-Time Forex Rate and Trade Feasibility**

**Scenario:** A corporate treasurer needs to know the live USD/INR exchange rate and confirm if the bank has enough liquidity to handle a large trade of $500,000 immediately.

#### **Naive Prompt (Non-ReAct) 👎**

The LLM lacks real-time market data and internal bank information, forcing it to be vague or provide stale, incorrect data.

In [ ]:
naive_prompt_react_2 = "What's the current USD/INR exchange rate, and can I execute a $500,000 trade right now?"

# execute_prompt(
#     "{query}",
#     "1.2 Naive FX Inquiry",
#     query=naive_prompt_react_2
# )

#### **ReAct Implementation 👍**

We provide the agent with tools to fetch live data and check internal systems, allowing it to give a complete, actionable answer.

In [ ]:
# --- New Tool Definitions ---
@tool
def get_forex_rate(base_currency: str, quote_currency: str) -> float:
    """
    Returns the real-time exchange rate for a given currency pair.
    Use this to get current foreign exchange rates.
    """
    print(f"--- TOOL EXECUTED: get_forex_rate({base_currency}, {quote_currency}) ---")
    # Mock live data for demonstration
    if base_currency == "USD" and quote_currency == "INR":
        return 83.54
    return 0.0

@tool
def check_trade_feasibility(currency: str, amount: float) -> str:
    """
    Checks if a trade of a certain size is feasible based on the bank's current liquidity and client limits.
    Use this to verify if a large trade can be executed.
    """
    print(f"--- TOOL EXECUTED: check_trade_feasibility({currency}, {amount}) ---")
    if currency == "USD" and amount <= 1000000:
        return "Feasible: The trade is within the current liquidity limits for this client."
    else:
        return "Not Feasible: The trade amount exceeds the bank's current single-trade limit. Please contact the trading desk."

# --- Agent Creation & Execution ---
# We can reuse the same agent setup from the previous example, just with new tools.
prompt = hub.pull("hwchase17/react")
tools_fx = [get_forex_rate, check_trade_feasibility]
agent_fx = create_react_agent(llm, tools_fx, prompt)
agent_executor_fx = AgentExecutor(agent=agent_fx, tools=tools_fx, verbose=True)

# query = "What's the current USD/INR exchange rate, and can I execute a $500,000 trade right now?"
# response = agent_executor_fx.invoke({"input": query})
# print("--- FINAL AGENT RESPONSE ---")
# print(response['output'])

-----

## **2. Chain-of-Verification (CoVe): Minimizing Hallucinations**

### **Background**

Factual accuracy is non-negotiable in banking. **Chain-of-Verification (CoVe)** is a technique designed to make an LLM double-check its own work, significantly reducing the risk of generating plausible-sounding but incorrect information (hallucinations).

  * **How it works:** It's a deliberate, multi-step process.
    1.  **Draft Response:** The LLM generates an initial answer to the query.
    2.  **Plan Verifications:** The model reviews its own draft and generates a list of factual claims that need to be checked.
    3.  **Execute Verifications:** The model answers these verification questions *independently*, which prevents the initial draft from biasing the check.
    4.  **Final Verified Response:** The model revises the initial draft based on the outcome of the verifications, correcting any inaccuracies.
  * **Strengths:** Drastically improves the factual grounding and reliability of generated text. It's an essential technique for high-stakes content generation.
  * **Banking Use Cases:** Summarizing complex regulatory documents (like RBI circulars), generating financial reports, creating internal policy documentation, or drafting market analysis where every claim must be backed by data.

### **Use Case 2.1: Summarizing a Regulatory Guideline**

**Scenario:** A compliance officer needs a quick, accurate summary of a new (fictional) RBI guideline on digital lending.

In [ ]:
rbi_guideline_text = """
RBI Circular No. 2025/11/A-34
Subject: Guidelines on Digital Lending for Non-Banking Financial Companies (NBFCs)

1. Introduction: To ensure customer protection and data security, all NBFCs engaged in digital lending must adhere to the following.
2. Loan Disbursal: All loan disbursals must be executed directly from the NBFC's bank account to the borrower's bank account. No third-party pool accounts are permitted. This rule is effective from December 1, 2025.
3. Data Privacy: Only minimal, necessary customer data may be collected. Explicit customer consent is required for any data sharing. Biometric data cannot be stored.
4. Applicability: These guidelines apply to all NBFCs. Regional Rural Banks (RRBs) are currently exempt.
"""

#### **Naive Prompt 👎**

A simple summarization request can lead to subtle but critical errors of omission or misinterpretation.

In [ ]:
# naive_prompt_cove_1 = "Summarize the key points of the following RBI guideline: \n\n{document}"
# naive_summary = execute_prompt(
#     naive_prompt_cove_1,
#     "2.1 Naive Guideline Summary",
#     document=rbi_guideline_text
# )


**Critique:** The summary is mostly correct, but it makes one critical mistake: it claims the rules apply to "all financial institutions," when the text clearly exempts Regional Rural Banks (RRBs).

#### **CoVe Implementation 👍**

We simulate the CoVe process using a sequence of prompts to force self-correction.

In [ ]:
# --- CoVe Step 1: Draft Initial Response ---
step1_prompt = "Draft a summary of the key points of the following RBI guideline: \n\n{document}"
draft_summary = execute_prompt(
    step1_prompt,
    "CoVe Step 1: Draft Summary",
    document=rbi_guideline_text
)

# --- CoVe Step 2: Plan Verifications ---
step2_prompt = """
Based on the following summary, generate a list of verification questions to fact-check its claims against the original document.

Summary:
{summary}
"""
verification_plan = execute_prompt(
    step2_prompt,
    "CoVe Step 2: Plan Verifications",
    summary=draft_summary
)

# --- CoVe Step 3: Execute Verifications (Simulated) ---
# In a real system, you'd answer these questions independently. Here, we bundle them.
step3_prompt = """
Answer the following questions based *only* on the provided RBI document.

Document:
{document}

Questions:
{questions}
"""
verification_results = execute_prompt(
    step3_prompt,
    "CoVe Step 3: Execute Verifications",
    document=rbi_guideline_text,
    questions=verification_plan
)

# --- CoVe Step 4: Generate Final Verified Response ---
step4_prompt = """
You have an initial draft summary and a set of verified facts.
Your task is to rewrite the summary, correcting any inaccuracies or omissions based on the verified facts.

Initial Summary:
{summary}

Verified Facts (Q&A):
{verifications}

Final Verified Summary:
"""
final_summary = execute_prompt(
    step4_prompt,
    "CoVe Step 4: Final Verified Summary",
    summary=draft_summary,
    verifications=verification_results
)

This final version correctly captures the nuance of the exemption, which the naive prompt missed.

### **Chain-of-Verification (CoVe) Use Case 2.2: Generating Factual Stock Analysis**

**Scenario:** A junior equity analyst needs to write a short, factually precise summary of a company's quarterly performance for an internal client newsletter. The summary must not contain any speculation.

In [ ]:
# Fictional quarterly data points for "Innovate Corp."
stock_data = """
- Company: Innovate Corp.
- Revenue: ₹150 Crores (an increase of 12% Year-over-Year).
- Net Profit: ₹25 Crores (an increase of 8% Year-over-Year).
- Key Highlight: Successfully launched new AI platform 'Cognito'.
- Stock Price Movement (last quarter): -5%.
- Analyst Consensus Rating: 'Hold'.
"""

#### **Naive Prompt 👎**

The model might try to be "helpful" by inventing a narrative or a cause-and-effect relationship that isn't supported by the facts provided.

In [ ]:
# naive_prompt_cove_2 = "Write a short paragraph analyzing Innovate Corp's latest quarterly results based on these data points: \n\n{data}"
# naive_analysis = execute_prompt(
#     naive_prompt_cove_2,
#     "2.2 Naive Stock Analysis",
#     data=stock_data
# )

**Critique:** The phrase "market reacted skeptically" directly and incorrectly links the stock drop to the results, a causal relationship not present in the source data. This is speculation, not factual reporting.

#### **CoVe Implementation 👍**

The CoVe process forces the model to check its own claims against the source data, stripping out any speculative connections.

In [ ]:
# --- CoVe Step 1: Draft Initial Response ---
step1_prompt = "Draft a short paragraph analyzing Innovate Corp's latest quarterly results based on these data points: \n\n{data}"
draft_analysis = execute_prompt(
    step1_prompt,
    "CoVe Step 1: Draft Analysis",
    data=stock_data
)

# --- CoVe Step 2: Plan Verifications ---
step2_prompt = """
Based on the following draft analysis, generate a list of verification questions to fact-check its claims against the original data points.

Draft Analysis:
{analysis}
"""
verification_plan_2 = execute_prompt(
    step2_prompt,
    "CoVe Step 2: Plan Verifications",
    analysis=draft_analysis
)

# --- CoVe Step 3: Execute Verifications ---
step3_prompt = """
Answer the following questions based *only* on the provided data points. Do not infer or speculate.

Data Points:
{data}

Questions:
{questions}
"""
verification_results_2 = execute_prompt(
    step3_prompt,
    "CoVe Step 3: Execute Verifications",
    data=stock_data,
    questions=verification_plan_2
)

# --- CoVe Step 4: Generate Final Verified Response ---
step4_prompt = """
You have a draft analysis and a set of verified facts. Your task is to rewrite the analysis, ensuring every statement is directly supported by the verified facts. Remove any speculation or causal links that are not explicitly mentioned in the data.

Draft Analysis:
{analysis}

Verified Facts (Q&A):
{verifications}

Final Verified Analysis:
"""
final_analysis = execute_prompt(
    step4_prompt,
    "CoVe Step 4: Final Verified Analysis",
    analysis=draft_analysis,
    verifications=verification_results_2
)

This final version is factually impeccable. It presents the different data points without inventing a narrative to connect them, which is exactly what's required for unbiased financial reporting.